In [ ]:
import requests
import gzip
import shutil
import pandas
import os
import datetime
import pymongo
import time

# Começa a contagem do tempo
tempo_inicio = time.time()

# Cria uma conexão com o mongo
myclient = pymongo.MongoClient(MONGO_HOST)
mydb = myclient[MONGO_DB]
mycol = mydb[MONGO_COLLECTION]

#Pega o datetime corrente
now = datetime.datetime.now()

# define the name of the directory to be created
path = "tmp/"+ str(now.year) +"/"+ str(now.month) +"/"

try:
    os.makedirs(path)
except OSError:
    print ("Não foi possível criar o diretório: %s" % path)
        
print('Iniciando o download dos dados abertos do sagres...')

# Fazendo a requisição e salvando o arquivo
r = requests.get('https://dados.tce.pb.gov.br/TCE-PB-Portal-Gestor-Licitacoes_Propostas.txt.gz')

print('Salvando arquivo...')

# Salvando arquivo
with open(path + 'sagres_import.txt.gz', 'wb') as f:
    f.write(r.content)

# Retrieve HTTP meta-data
if r.status_code == 200 :
    print('Arquivo baixado com sucesso!')
    #print(r.headers['content-type'])

# Descompacta o arquivo e salva como CSV
with gzip.open(path + 'sagres_import.txt.gz', 'rb') as f_in:
    with open(path + 'sagres_import.csv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)


# Carrega os dados do sagres        
data_sagres = pandas.read_csv(path + 'sagres_import.csv', delimiter='|', encoding = 'UTF-8', low_memory=False)
data_sagres = data_sagres.loc[
    (data_sagres['nome_esfera_jurisdicionado'] == 'Municipal') &
    (data_sagres['nome_municipio'] == 'João Pessoa') &
    (data_sagres['situacao_proposta'] == 'Vencedora') &
    (data_sagres['nome_tipo_jurisdicionado'].isin(['Órgão','Prefeitura','Fundo']))
]

# Carrega os dados dos contratos do portal da transparencia
data_contratos_jp = requests.get('https://transparencia.joaopessoa.pb.gov.br:8080/licitacoes/contratos?data_assinatura_periodo=2020-1-1&data_assinatura_periodo=2021-4-30')
data_contratos_jp = pandas.DataFrame.from_dict(data_contratos_jp.json())

# Percorrer as linhas dos dados do sagres
for i, s in data_sagres.iterrows():
    # Extrai o ano do numero do processo
    ano_licitacao_sagres = int(s.numero_licitacao[6:10])
    
    # Só considera licitações entre 2017 e 2020
    if (ano_licitacao_sagres >= 2020) & (ano_licitacao_sagres <= 2021) :
        #Padroniza o numero da licitação do sagres
        numero_licitacao_sagres = s.numero_licitacao.replace("/","")
        numero_licitacao_sagres = int(numero_licitacao_sagres)
        numero_licitacao_sagres = '{:0>9}'.format(numero_licitacao_sagres)
        
        # Inicializa o JSON
        contratos_dict = {}
        
        # Busca pelo contrato
        for j, r in data_contratos_jp.iterrows():
            #Padroniza o numero da licitação do portal da transparencia
            numero_licitacao_portal = int(r.numero_licitacao)
            numero_licitacao_portal = '{:0>9}'.format(r.numero_licitacao)
        
            if (numero_licitacao_sagres == numero_licitacao_portal) & (s.cpf_cnpj_proponente == r.cpf_cnpj_fornecedor) & (s.valor_proposta == r.valor) :
                #print("Achou o contrato da licitação: "+numero_licitacao_sagres)
                contratos_dict = {
                    "numero_contrato": r.numero_contrato, 
                    "objeto_contrato": r.objeto_contrato, 
                    "inicio_vigencia": r.inicio_vigencia,
                    "fim_vigencia": r.fim_vigencia,
                    "contratante": r.contratante,
                    "fornecedor": s.nome_proponente,
                    "cpf_cnpj_fornecedor": s.cpf_cnpj_proponente,
                    "valor_proposta": s.valor_proposta
                }
                break;            
        else: # Caso não encontre nada no for
            #print("Não achou o contrato da licitação: "+numero_licitacao_sagres)
            contratos_dict = {
                "fornecedor":s.nome_proponente,
                "cpf_cnpj_fornecedor":s.cpf_cnpj_proponente,
                "valor_proposta":s.valor_proposta,
            }        
                
        # Busca no banco se existe uma licitação com essas características
        mydoc = mycol.find_one({ 
            "numero_licitacao":numero_licitacao_sagres,
            "valor_licitado_licitacao":s.valor_licitado_licitacao,
            "data_homologacao_licitacao":s.data_homologacao_licitacao,
            "nome_modalidade_licitacao":s.nome_modalidade_licitacao,
            "nome_jurisdicionado":s.nome_jurisdicionado
        })
        
        # Caso não tenha inserido ainda a licitação
        if str(mydoc) == "None" :            
            mycol.insert_one({
                "protocolo_licitacao":s.protocolo_licitacao,
                "numero_licitacao":numero_licitacao_sagres,
                "nome_modalidade_licitacao":s.nome_modalidade_licitacao,
                "nome_municipio":s.nome_municipio,
                "cd_ugestora": s.cd_ugestora,
                "jurisdicionado_id": s.jurisdicionado_id,
                "nome_jurisdicionado":s.nome_jurisdicionado,
                "objeto_licitacao":s.objeto_licitacao,
                "valor_estimado_licitacao":s.valor_estimado_licitacao,
                "valor_licitado_licitacao":s.valor_licitado_licitacao,
                "data_homologacao_licitacao":s.data_homologacao_licitacao,
                "ano_homologacao_licitacao":s.ano_homologacao_licitacao,
                "nome_estagio_processual_licitacao":s.nome_estagio_processual_licitacao,
                "nome_setor_atual_licitacao":s.nome_setor_atual_licitacao,
                "entidade_governamental":"Prefeitura Municipal de João Pessoa",
                "url":s.url,
                "contratos":[contratos_dict]
            })       
        else : # Caso já exista, acrescentar apenas o contrato
            mydoc["contratos"].append(contratos_dict)
            mycol.update_one( {"_id":mydoc["_id"]}, { "$set": mydoc })
            
tempoExec = time.time() - tempo_inicio
print("Tempo de execução: {} segundos".format(tempoExec))